# Recherche d'information dans la littérature scientifique

**MOD 7.2 (Introduction à la sciences des données)**, BE séances 4, 5, 6

**Enseignants :** Julien Velcin (CM, BE), Erwan Versmée (BE)

Le projet qu'on vous demande de réaliser est un **moteur de recherche** qui, étant donnée une publication, retourne les articles qui en sont le plus proche sémantiquement.
Pour simplifier le problème, pour une **requête donnée** (*query*), càd un article, on vous fournit 5 articles cités par celui-ci (les exemples positifs) et environ 25 articles choisis aléatoirement dans la base (les exemples négatifs).

La tâche consiste alors à construire un algorithme capable de **retourner les 5 citations et écarter les autres**.

Notez, qu'en plus du titre, d'**autres informations** susceptibles d'être utiles vous sont fournies : court résumé textuel, auteur(s) de l’article, année de publication, références bibliographiques, quels autres articles le citent.

**Exemple :**

Titre de l'article ciblé (notre requête, pour simplifier) : *"Bearish-Bullish Sentiment Analysis on Financial Microblogs"*

Quelques exemples positifs :

> "SemEval-2015 Task 11: Sentiment Analysis of Figurative Language in Twitter"
>
> "Text mining of news-headlines for FOREX market prediction: A Multi-layer Dimension Reduction Algorithm with semantics and sentiment"

Quelques exemples négatifs :

> "Analysis and Design of Average Current Mode Control Using a Describing-Function-Based Equivalent Circuit Model"
>
> "MVOR: A Multi-view RGB-D Operating Room Dataset for 2D and 3D Human Pose Estimation"


Votre moteur se basera sur un système *S* qui doit mesurer à quel point un article candidat *c* répond bien à la requête *q* à l'aide d'un **score d'appariement** S(q,c) : plus ce score est élevé, mieux l'article correspond à la requête.

Si *q* est la requête et *C* l'ensemble des candidats (environ 25 pour chaque requête), composés d'exemples positifs *C+* et négatifs *C-*, alors il faut que *S* retourne préférentiellement les exemples de *C+*, donc leur attribue un score important.


## Setup and Import Libraries

Ici se trouvent un certain nombre de librairies dont vous pourriez avoir besoin. Néanmoins, rien ne vous oblige à tout utiliser et vous pouvez ajouter vos propres librairies. Il est cependant important qu'il soit possible pour une tierce personne d'installer le nécessaire pour exécuter votre code.

In [14]:
import json
import numpy as np
import networkx as nx
from collections import defaultdict, Counter
from typing import Dict, List, Tuple
import os
import pickle
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')

# For embeddings and similarity computation
try:
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    print("Required libraries imported successfully!")
except ImportError as e:
    print(f"Missing library: {e}")
    print("Please install with: pip install sentence-transformers scikit-learn networkx")

np.random.seed(42)

Required libraries imported successfully!


Le code qui suit peut être utile pour réaliser des affichages à partir de la matrice Documents x Termes, libre à vous de l'utiliser.

In [ ]:
from scipy.sparse import find, csr_matrix
import matplotlib.pyplot as plt
import pandas as pd
from scipy.linalg import norm
from IPython.core.display import HTML

# des options permettent de limiter (ou non) le nombre de lignes/colonnes affichées
# par exemple :
# pd.set_option('display.max_rows', None)

# cette fonction permet d'afficher une "jolie" représentation du vecteur v
# ARGS :
#   v : le vecteur à afficher (par ex. une ligne de la matrice X)
#   features : le vocabulaire
#   top_n : le nombre de mots maximum à afficher
def print_feats(v, features, top_n = 30):
    _, ids, values = find(v)
    feats = [(ids[i], values[i], features[ids[i]]) for i in range(len(list(ids)))]
    top_feats = sorted(feats, key=lambda x: x[1], reverse=True)[0:top_n]
    return pd.DataFrame({"word" : [t[2] for t in top_feats], "value": [t[1] for t in top_feats]})   

# fonction qui permet d'afficher plusieurs tables pandas côte à côte (c'est cadeau)
def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "&emsp;"
        #output += "\xa0\xa0\xa0"
    display(HTML(output))

## 1. Chargement et prise en main des données

La première étape consiste bien entendu à charger les données en mémoire et aller voir comment celles-ci sont structurées. On vous fournit 3 fichiers pour commencer :

- *corpus.jsonl* : le corpus lui-même, composé de plus de 25k articles scientifiques
- *queries.jsonl* : l'ensemble des documents qui constituent les requêtes qui seront adressées à notre moteur. Ces documents proviennent du corpus, mais il peut s'agir de documents qui ne sont que cités ou citent d'autres documents et pour lesquels on n'a que très peu d'information.
- *valid.tsv* : l'ensemble des données nécessaires pour entraîner et/ou tester votre moteur

Voilà un extrait du fichier *corpus.jsonl* :

![corpus](./corpus.jpg "Title")

Voilà un extrait du fichier *queries.jsonl* :

![corpus](./queries.jpg "Title")

Voilà un extrait du fichier *valid.tsv* :

![corpus](./test.jpg "Title")

Par la suite, on vous fournira à priori un 4ème fichier (*test_final.tsv*) qui contiendra des nouvelles données sur lesquelles vous pourrez réaliser  des prédictions. Les identifiants des candidats sont toujours tirés de votre corpus d'articles.

Nous vous conseillons d'employer la libraire *json* pour charger les données des deux premiers fichiers et utiliser des dictionnaires afin de pouvoir accéder aux articles via leurs identifiants. Pour rappel, un fichier *jsonl* n'est rien d'autre qu'un fichier dans lequel chaque ligne est composé d'un objet json bien formaté.

In [ ]:
def load_corpus(file_path: str) -> Dict[str, Dict]:
    """
    TODO

    Load corpus data from JSONL file.
    Returns dictionary mapping document IDs to document data.
    """
   
def load_queries(file_path: str) -> Dict[str, Dict]:
    """
    TODO

    Load query data from JSONL file.
    Returns dictionary mapping query IDs to query data.
    """
    

def load_qrels(file_path: str) -> Dict[str, Dict[str, int]]:
    """
    TODO
    
    Load relevance judgments from TSV file.
    Returns dictionary mapping query IDs to candidate relevance scores.
    """
    

In [ ]:
# Load the dataset
print("Loading dataset...")
corpus = load_corpus('corpus.jsonl')
queries = load_queries('queries.jsonl')
qrels_valid = load_qrels('valid.tsv')


print(f"Loaded {len(corpus)} documents in corpus")
print(f"Loaded {len(queries)} queries")
print(f"Loaded relevance for {len(qrels_valid)} queries (dataset)")

## 2. Exploration des données et premier encodage

A présent, on vous propose de réaliser un certain nombre d'opérations pour mieux connaître vos données. Tout d'abord, il s'agit de calculer des statistiques simples telles que :

- Taille du corpus et du nombre de requêtes
- Nombre de paires requête / document
- Proportions de documents pertinents (non pertinents) par requête

Affichez ensuite un exemple de requête (vous pouvez vous limiter au titre de l'article), accompagnée d'exemples de candidats positifs et de candidats négatifs.

In [ ]:
# TODO


Ensuite, utilisez la librairie *scikit-learn* pour réaliser un **premier encodage des données** sous forme de matrice Documents * Termes.
Il suffit pour cela d'utiliser la classe *CountVectorizer* puis d'utiliser les fonction *fit* et *transform* pour (respectivement) construire le vocabulaire et la matrice elle-même.
Pour simplifier et accélérer les calculs, vous pouvez commencer en ne considérant que le titre des articles, mais rien ne vous empêche par la suite d'utiliser également le champ résumé.

Pour le moment, contentez-vous d'utiliser les paramètres par défaut, mais vous pourrez ensuite appliquer de **nombreux prétraitements** tels que la suppression des mots-outils ou la réduction de la taille du vocabulaire. Nous verrons ça dans la section suivante.

In [ ]:
# TODO 

En utilisant la fonction fournie *print_feats*, affichez les informations contenues dans quelques vecteurs de la base de données.

In [ ]:
# TODO


Affichez la distribution des mots les plus fréquemment employés dans le corpus. Pour cela, il suffit de faire la somme des occurrences par colonne, puis d'appeler la fonction d'affichage précédente. Ensuite, vous pouvez utiliser la librairie *matplotlib* pour afficher un histogramme qui permet d'avoir une représentation visuelle de cette distribution.

In [ ]:
# TODO


## 3. Comparaison de documents et premier moteur de recherche

Une manière classique de comparer des vecteurs creux (ce qui est le cas pour la matrice Documents x Termes) est d'utiliser la mesure du cosinus. Celle-ci calcule une **similarité entre deux textes** basée sur les mots en commun.

En utilisant la fonction *cosine_similarity* de la *librairie sklearn*, testez cette mesure sur différentes paires de texte. Vous pouvez également travailler avec les résumés des articles. N'hésitez pas à afficher les documents pour observer les mots qu'ils ont en commun.

In [ ]:
# TODO

Vous pouvez à présent écrire un **petit moteur de recherche** qui prend un ensemble de mots clefs, calcule un score pour chaque document de la base et retourne les 10 premieres résultats.

Pour cela, les différentes étapes sont les suivantes :

1. Construire le vecteur requête à partir du texte
1. Calculer le score (cosinus) entre la requête et tous les documents du corpus
1. Triez les résultats et affichez les 10 premiers articles

Pour la première étape, vous pouvez utiliser directement le "vectoriseur" pour construire le vecteur requête dans le même espace de vocabulaire que le corpus (fonction *transform*).

In [ ]:
# TODO

Pour en finir avec cette approche classique "creuse", vous pouvez essayer des variantes :

- Modifier les paramètres du "vectoriseur" : taille du vocabulaire, suppression des mots-outils, suppression des mots trop rares ou trop fréquents, stemming
- Utiliser une autre pondération, comme **TFxIDF**

## 4. Utiliser un meilleur encodeur de documents

L'encodage dans un espace de mots avec TF ou TFxIDF a de nombreuses limites. On utilise aujourd'hui des encodeurs spécialisés pour construire des **représentations denses** beaucoup plus adaptées (cf. cours).

La librairie [sentence-transformers](https://www.sbert.net) propose de très nombreux modèles près à l'emploi. Vous pouvez en choisir un par défaut (comme all-MiniLM-L6-v2, qui est assez rapide et permet d'obtenir des bonnes performances) ou vous laisser guider par les tests réalisés sur le benchmark [MTEB](https://huggingface.co/spaces/mteb/leaderboard).

Attention, suivant la puissance de votre machine (et la présence ou non d'un GPU), l'encodage peut prendre un certain temps. Nous vous conseillons de ne faire le calcul qu'**une seule fois** sur l'ensemble de votre corpus et de stocker les représentations (embeddings) ainsi construit dans un fichier.

Une alternative consiste à construire un index efficace via un **vector store** tel que [FAISS](https://faiss.ai/index.html).

In [ ]:
# TODO : construction des embeddings avec un sentence-transformer

# TODO : sauvegarde des embeddings dans un fichier pour réutilisation ultérieure

Cette partie permet de charger les embeddings en mémoire s'ils ont déjà été stockés dans un fichier.

In [ ]:
# TODO : charger les embeddings depuis un fichier si déjà existant

## 5. Nouveau moteur de recherche

A présent, l'objectif est double :

1. Implémenter un **moteur de recherche**, mais cette fois en vous basant sur les représentations denses construites à l'étape précédente.
2. **Evaluer** la pertinence de votre moteur grâce aux annotations fournies dans le fichier *valid.tsv* qui portent sur les requêtes de *query.tsv* : pour une requête donnée, votre système doit retourner préférentiellement les articles pertinents (score noté 1) en comparaison des articles non pertinents (score noté 0).

Commencez par implémenter le moteur de recherche proprement dit : à partir d'une requête, le système doit trier les 25 documents candidats fournis.

In [ ]:
# TODO

Prenez quelques requêtes en exemple et affichez le résultat de votre moteur. On doit pouvoir vérifier si les 5 premiers résultats sont bien pertinents (score de 1).

In [ ]:
# TODO

A présent, faites tourner le moteur de recherche sur l'ensemble des requêtes du fichier et calculez des indicateurs de qualité.

Il s'agit à minima de :
- précision
- rappel
- f-mesure
- AUC

In [ ]:
# TODO

## 6. Intermède : exploration avec des modèles thématiques

Cette partie est purement exploratoire. Elle consiste à tester au moins un algorithme d'extraction de thématique, tel que LDA. N'hésitez pas à vous référer au notebook associés au cours sur cette partie.

Rien ne vous empêche de tester d'autres algorithmes, parmi lesquels ceux cités durant le cours.

In [ ]:
# TODO

## 7. Construction du graphe de citations

A présent, l'objectif est de construire le graphe de citations et de l'utiliser pour essayer d'améliorer les résultats de notre système de recherche d'information.

Utilisez la librairie Networkx pour construire le graphe à partir de vos fichiers de données. N'hésitez pas à vous référer au notebook fourni associés au cours.

In [ ]:
# TODO

Calculez des indicateurs élémentaires sur ce graphe, par exemple :

- Nombre de noeuds
- Nombre d'arcs
- Densité du graphe
- Degrés entrants et sortants (moyenne, variance)

In [ ]:
# TODO

Pour finir, calculez des indicateurs de centralité afin de faire ressortir les articles qui semblent les plus influents. Essayez de comparer, même qualitativement, les résultats qui peuvent être différents en fonction de la mesure.

In [ ]:
# TODO

## 8. Construire des meilleures représentations pour les noeuds

L'idée principale consiste à utiliser la structure pour améliorer la représentation vectorielle des noeuds / documents. Le plus simple consiste à utiliser les représentations des voisins directs des noeuds, par ex. via une moyenne qu'il est possible de pondérer.

In [ ]:
# TODO

Une fois ces meilleures représentations calculées, vous pouvez tester si elles permettent d'améliorer les résultats de votre moteur de recherche.

In [1]:
# TODO

## 9. Ce qui est attendu

Dans ce projet, le minimum attendu consiste à développer une solution qui teste au moins 1 méthode pour chacun des trois manières de représenter les données, à savoir :
- une approche creuse
- une approche dense
- une approche qui utilise l'information de structure

Néanmoins, il est fortement conseillé de tester *plusieurs* variantes pour chacune de ces manières. Ainsi, pour l'approche creuse, vous pouvez essayer plusieurs techniques de prétraitement et plusieurs schémas de pondération. Pour l'approche dense, vous pouvez essayer plusieurs encodeurs, utiliser un vector store (comme FAISS).

Un document précisant les **consignes** pour ce projet sera fourni en complément de ce notebook.

## 10. Pour aller plus loin

De nombreux raffinements peuvent être envisagés, parmi lesquels :

- Combinaison des approches creuses et denses
- Utilisation d'approches neuronales pour combiner le texte et la structure (par ex. GCN, GNN)
- Utilisation d'approches neuronales pour améliorer la recherche d'information (par ex. COLBERT)